In [1]:
# Load packages and read in raw data

import pandas as pd
import numpy as np

dete = pd.read_csv('../dete-exit-survey-january-2014.csv', 
                    na_values='Not Stated' #replace 'Not Stated' values with nulls
                  )
tafe = pd.read_csv('../tafe-employee-exit-survey-access-database-december-2013.csv', 
                   encoding='iso-8859-1'
                  )

# Remove unnecessary columns

dete_updated = dete.drop(dete.columns[28:49], axis=1)
tafe_updated = tafe.drop(tafe.columns[17:66], axis=1)

In [2]:
# Set column and row display maximums

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Research Questions
- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?

In [3]:
# Standardize dete_updated column names

dete_updated.columns = dete_updated.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('/', '_')
dete_updated.rename({'separationtype':'separation_type'}, inplace=True, axis=1)

In [4]:
# Standardize tafe_updated column names

pattern = r'[-\.?()/]'
tafe_updated.columns = tafe_updated.columns.str.replace(pattern, '').str.lower().str.strip()
tafe_updated.columns = tafe_updated.columns.str.replace('\s+', ' ').str.replace(' ', '_')

In [5]:
tafe_column_name_map = {'record_id':'id',
                        'reason_for_ceasing_employment':'separation_type',
                        'cessation_year':'cease_date',
                        'workarea':'position',
                        'employment_type_employment_type':'employment_status',
                        'contributing_factors_career_move_public_sector':'career_move_to_public_sector',
                        'contributing_factors_career_move_private_sector':'career_move_to_private_sector',
                        'contributing_factors_interpersonal_conflict':'interpersonal_conflicts',
                        'contributing_factors_job_dissatisfaction':'job_dissatisfaction',
                        'contributing_factors_maternityfamily': 'maternity_family',
                        'contributing_factors_ill_health':'ill_health',
                        'gender_what_is_your_gender':'gender',
                        'currentage_current_age': 'age',
                        'lengthofserviceoverall_overall_length_of_service_at_institute_in_years': 'length_of_service'}

tafe_updated = tafe_updated.rename(tafe_column_name_map, axis=1)
tafe_updated.drop(['institute', 
                   'contributing_factors_career_move_selfemployment', 
                   'contributing_factors_dissatisfaction',
                   'contributing_factors_study',
                   'contributing_factors_travel',
                   'contributing_factors_other',
                   'contributing_factors_none',
                   'classification_classification',
                   'lengthofservicecurrent_length_of_service_at_current_workplace_in_years'], axis=1, inplace=True)

In [6]:
dete_updated.drop(['role_start_date',
                   'classification',
                   'region',
                   'business_unit',
                   'dissatisfaction_with_the_department',
                   'physical_work_environment',
                   'lack_of_recognition',
                   'lack_of_job_security',
                   'work_location',
                   'employment_conditions',
                   'relocation',
                   'study_travel',
                   'traumatic_incident',
                   'work_life_balance',
                   'workload',
                   'none_of_the_above',
                   'aboriginal',
                   'torres_strait',
                   'south_sea',
                   'disability',
                   'nesb'], axis=1, inplace=True)

In [7]:
# Standardize date format of cease_date in dete - removing month from some some entries

dete_updated['cease_date'] = dete_updated['cease_date'].str.replace(r'[0-9]{2}/', '')

In [8]:
# Create length of service column

dete_updated['cease_date'] = pd.to_numeric(dete_updated['cease_date'])
dete_updated['length_of_service'] = dete_updated['cease_date'] - dete_updated['dete_start_date']
dete_updated.drop('dete_start_date', inplace=True, axis=1)

In [9]:
# Change resignation reasons to single type - Resignation

dete_updated.loc[dete_updated['separation_type'].str.contains('Resignat'), 'separation_type'] = 'Resignation'
dete_updated.loc[dete_updated['separation_type'].str.contains('Retirement'), 'separation_type'] = 'Retirement'

In [11]:
# Create function to convert float to categorical variable

def create_year_categories(element):
    if element == 0:
        return 'Less than 1 year'
    elif (element > 0) & (element < 3):
        return '1-2'
    elif (element > 2) & (element < 5):
        return '3-4'
    elif (element > 4) & (element < 7):
        return '5-6'
    elif (element > 6) & (element < 11):
        return '7-10'
    elif (element > 10) & (element < 21):
        return '11-20'
    elif element > 20:
        return 'More than 20 years'

In [12]:
# Convert float to categorical variable

dete_updated['length_of_service'] = dete_updated['length_of_service'].apply(create_year_categories)

In [13]:
# Create function to convert values to boolean

def convert_to_bool(element):
    if element == '-':
        return False
    else:
        return True

In [14]:
# Convert values from selected columns to boolean

columns_to_convert_to_boolean = ['career_move_to_public_sector',
                                 'career_move_to_private_sector',
                                 'ill_health',
                                 'maternity_family',
                                 'job_dissatisfaction',
                                 'interpersonal_conflicts']

for column in columns_to_convert_to_boolean:
    tafe_updated[column] = tafe_updated[column].apply(convert_to_bool)

In [15]:
# Create function to convert age column of dete_updated

def convert_age_cat(element):
    if (element == '56-60') or (element == '61 or older'):
        return '56 or older'
    else:
        return element

In [16]:
# Convert age column of dete_updated

dete_updated['age'] = dete_updated['age'].apply(convert_age_cat)

In [17]:
# Create function to update values in tafe_updated.age

def update_age_cat_spacing(element):
    tafe_age_vals = tafe_updated.age.value_counts().index[2:-1]
    if element == tafe_age_vals[0]:
        return '41-45'
    elif element == tafe_age_vals[1]:
        return '46-50'
    elif element == tafe_age_vals[2]:
        return '31-35'
    elif element == tafe_age_vals[3]:
        return '36-40'
    elif element == tafe_age_vals[4]:
        return '26-30'
    elif element == tafe_age_vals[5]:
        return '21-25'
    else:
        return element
    
# Update values in age column

tafe_updated['age'] = tafe_updated['age'].apply(update_age_cat_spacing)

In [18]:
# Create variable in each df to distinguish which df it came from

dete_updated['dataset'] = 'dete'
tafe_updated['dataset'] = 'tafe'

# Vertically concatenate dete_updated and tafe_updated

combined = pd.concat([dete_updated, tafe_updated], ignore_index=True, sort=False)

In [19]:
for i in combined.columns[1:]:
    print(combined[i].value_counts(dropna=False))

Resignation                 651
Retirement                  495
Contract Expired            161
Retrenchment/ Redundancy    104
Other                        49
Termination                  38
Transfer                     25
NaN                           1
Name: separation_type, dtype: int64
 2012.0    589
 2013.0    465
 2011.0    268
 2010.0    105
 2014.0     51
NaN         41
 2009.0      4
 2006.0      1
Name: cease_date, dtype: int64
Non-Delivery (corporate)                                   432
Teacher                                                    324
Delivery (teaching)                                        270
Teacher Aide                                               137
Public Servant                                             126
Cleaner                                                     97
Head of Curriculum/Head of Special Education                38
Schools Officer                                             24
School Administrative Staff                          

*Next Steps*
- Determine where length_of_service and age NaNs came from
- Remove position column
- Merge casual and contract/casual in employment status column
- Drop rows with older cease date?
- What to do with NaNs?